<a href="https://colab.research.google.com/github/kootr/ml-study-session/blob/main/20220730_reccomendation_mf/bq_recommendation_mf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

if os.getenv("IS_TESTING"):
    ! touch /builder/home/.local/lib/python3.9/site-packages/google_api_core-2.7.1.dist-info/METADATA

In [3]:
client = bigquery.Client(location="US", project="ml-session")

# Example: Send a query to a Public dataset

In [2]:
import os
import sys

IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

KeyboardInterrupt: ignored

In [4]:
query = """
SELECT
  vendor_id,
  passenger_count,
  trip_distance,
  rate_code,
  payment_type,
  total_amount,
  tip_amount
FROM
  `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
WHERE tip_amount >= 0
LIMIT 100
"""
query_job = client.query(
    query,
     location="US",
)

df = query_job.to_dataframe()
df.head(5)

Forbidden: ignored

In [ ]:
df.passenger_count.describe()

In [ ]:
df.astype({'total_amount': float}).total_amount.describe()

# TMDB 5000 Movie Dataset
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [ ]:
# Reffered from the following URLs
# https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
# https://www.kaggle.com/code/rafffael/geting-started-movie-recommendation-with-eda-demo

# Load data from Cloud Strage

In [ ]:
from google.cloud import storage
gcs_client = storage.Client(project=' ml-session')

In [ ]:
import pandas as pd 
import numpy as np
# Get the uri from the left pane
df_credits=pd.read_csv('gs://ml_session9/TMDB/tmdb_5000_credits.csv')
df_movies=pd.read_csv("gs://ml_session9/TMDB/tmdb_5000_movies.csv")

In [ ]:
df_credits.head()

In [ ]:
df_movies.head(1)

The first dataset contains the following features:-

- movie_id - A unique identifier for each movie.
- cast - The name of lead and supporting actors.
- crew - The name of Director, Editor, Composer, Writer etc.

The second dataset has the following features:-
- budget - The budget in which the movie was made.
- genre - The genre of the movie, Action, Comedy ,Thriller etc.
- homepage - A link to the homepage of the movie.
- id - This is infact the movie_id as in the first dataset.
- keywords - The keywords or tags related to the movie.
- original_language - The language in which the movie was made.
- original_title - The title of the movie before translation or adaptation.
- overview - A brief description of the movie.
- popularity - A numeric quantity specifying the movie popularity.
- production_companies - The production house of the movie.
- production_countries - The country in which it was produced.
- release_date - The date on which it was released.
- revenue - The worldwide revenue generated by the movie.
- runtime - The running time of the movie in minutes.
- status - "Released" or "Rumored".
- tagline - Movie's tagline.
- title - Title of the movie.
- vote_average - average ratings the movie recieved.
- vote_count - the count of votes recieved.
Let's join the two dataset on the 'id' column

In [ ]:
df_credits.columns = ['id','title2','cast','crew']

In [ ]:
df_movies= df_movies.merge(df_credits,on='id')

In [ ]:
df_movies.head(1)

In [ ]:
!pip install ipywidgets --user

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
report = ProfileReport(df_movies)

In [ ]:
# convert date appropriate format
df_movies["release_date"] = pd.to_datetime(df_movies['release_date'])
df_movies['release_year'] = df_movies['release_date'].dt.year
df_movies['release_month'] = df_movies['release_date'].dt.month_name()
del df_movies["release_date"]

In [ ]:
import json
# transform all columns which had json string into json format
# several columns has id and name
# leave only name for them beacause id is not so important in this case
json_columns = {'cast', 'crew', 'genres', 'keywords', 'production_countries', 
                'production_companies', 'spoken_languages'}

for c in json_columns:
    df_movies[c] = df_movies[c].apply(json.loads)
    if c != "crew": # We need other information besides the name
        df_movies[c] = df_movies[c].apply(lambda row: [x["name"] for x in row])

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df_movies.head(1)

In [ ]:
from functools import partial

In [ ]:
# create director writer and producer columns 
def get_job(job, row):
    person_name = [x['name'] for x in row if x['job']==job] 
    return person_name[0] if len(person_name) else np.nan

df_movies["director"] = df_movies["crew"].apply(partial(get_job, "Director"))
df_movies["writer"]   = df_movies["crew"].apply(partial(get_job, "Writer"))
df_movies["producer"] = df_movies["crew"].apply(partial(get_job, "Producer"))
del df_movies["crew"]

# create profit column
df_movies["profit"] = df_movies["revenue"] - df_movies["budget"]

In [ ]:
# fill some missing values by most frequent one
for col in ["runtime", "release_year", "release_month"]:
    df_movies[col] = df_movies[col].fillna(df_movies[col].mode().iloc[0])

In [ ]:
df_movies.head(1)

In [ ]:
df_movies.head(1).keywords.sum()[:5]

In [ ]:
# import numpy as np
# import os
# import pickle
# from pywaffle import Waffle
# import scipy
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# pd.set_option("display.max_columns", None)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
top_movie_num = 10
top_movies_based_on = ["budget", "revenue", "profit", "vote_average", "vote_count", "popularity"]

fig, ax = plt.subplots(len(top_movies_based_on)//3, 3, figsize=(30,14))
colors = plt.cm.get_cmap('viridis', top_movie_num)

for i, col in enumerate(top_movies_based_on):
    top_movies_by = df_movies.sort_values(by=[col], ascending=False).head(top_movie_num)
    r, c = i//3, i%3
    ax[r][c].barh(top_movies_by["title"], top_movies_by[col], color=colors.colors)
    ax[r][c].set_title(f"top movies based on {col}", fontsize=24, pad=20)
    ax[r][c].tick_params(axis='both', which='major', labelsize=16)
    ax[r][c].invert_yaxis()    
    
plt.suptitle('Top movies based on some features',fontsize=30, y=1)
fig.tight_layout()

In [ ]:
!pip install squarify --user

In [ ]:
from collections import defaultdict, Counter
import squarify

In [ ]:
df_movies["keywords"][0][:5]

In [ ]:
df_movies["genres"][0]

In [ ]:
df_movies["original_language"][0]

In [ ]:
# plot genre frequncy with treemap 
genres_flatten = sum(df_movies["genres"].values, []) # genres are list of lists
genres_info = Counter(genres_flatten)

plt.figure(figsize=(15, 10))
plt.title("Genres", fontsize=25, pad=20)
squarify.plot(genres_info.values(), label=genres_info.keys(), text_kwargs={'fontsize':12}, bar_kwargs={'alpha':.7}, pad=True)
plt.axis("off");

In [ ]:
top_genres_num = 15
genres_flatten = sum(df_movies["genres"].values, []) # genres are list of lists
genres_info = Counter(genres_flatten)
top_genres = dict(genres_info.most_common(top_genres_num))

plt.figure(figsize=(20, 6))
plt.xticks(rotation=85, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel("Nb. of occurences", fontsize=18, labelpad=10)
plt.bar(top_genres.keys(), top_genres.values(), align='center', color=plt.cm.get_cmap('plasma', top_genres_num).colors)
# plt.bar(top_keywords.keys(), top_keywords.values(), align='center', color=plt.cm.get_cmap('plasma', top_keywords_num).colors)
plt.title("Genres", fontsize=25, pad=20)
plt.show()

In [ ]:
top_keywords_num = 30
keywords_flatten = sum(df_movies["keywords"].values, []) # keywords are list of lists
keywords_info = Counter(keywords_flatten)
top_keywords = dict(keywords_info.most_common(top_keywords_num))

plt.figure(figsize=(20, 6))
plt.xticks(rotation=85, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel("Nb. of occurences", fontsize=18, labelpad=10)
plt.bar(top_keywords.keys(), top_keywords.values(), align='center', color=plt.cm.get_cmap('plasma', top_keywords_num).colors)
plt.title("Keywords popularity", fontsize=25, pad=20)
plt.show()

In [ ]:
df_movies["original_language"].value_counts()[:5]

In [ ]:
plt.figure(figsize=(20, 6))
plt.xticks(rotation=85, fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel("Nb. of occurences", fontsize=18, labelpad=10)

plt.bar(df_movies["original_language"].value_counts()[:10].index,
        df_movies["original_language"].value_counts()[:10].values,
        align='center', color=plt.cm.get_cmap('plasma', top_keywords_num).colors)

plt.title("N of original language", fontsize=25, pad=20)
plt.show()

In [ ]:
df_movies["production_countries"].values

In [ ]:
top_contries_num = 10
production_contries_flatten = sum(df_movies["production_countries"].values, []) # production_countries are list of lists
production_contries_info = Counter(production_contries_flatten)
top_production_contries = production_contries_info.most_common(top_contries_num)

plt.figure(figsize=(30, 8))
plt.xticks(rotation=65, fontsize=15)
plt.ylabel("Nb. of occurences", fontsize=18)
plt.title("Production countries", fontsize=25, pad=20)
plt.bar(dict(top_production_contries).keys(),
        dict(top_production_contries).values(),
        color=plt.cm.get_cmap('cividis', top_contries_num).colors);

In [ ]:
top_companies_num = 15
production_companies_flatten = sum(df_movies["production_companies"].values, []) # production_companies are list of lists
production_companies_info = Counter(production_companies_flatten)
top_production_companies = production_companies_info.most_common(10)


plt.figure(figsize=(30, 8))
plt.xticks(rotation=65, fontsize=15)
plt.ylabel("Nb. of occurences", fontsize=18)
plt.title("Production countries", fontsize=25, pad=20)
plt.bar(dict(top_production_companies).keys(),
        dict(top_production_companies).values(),
        color=plt.cm.get_cmap('cividis', top_companies_num).colors);

# fig = plt.figure(
#     FigureClass=Waffle, 
#     rows=30, 
#     values=dict(top_production_companies), 
#     legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1)},
#     icons='child', icon_size=14, 
#     icon_legend=True,
#     figsize=(24,8)
# )

In [ ]:
!pip install wordcloud --user
from wordcloud import WordCloud

In [ ]:
df_movies["cast"].values[0][:5]

In [ ]:
# plot unique cast frequency
cast_flatten = sum(df_movies["cast"].values, []) # cast are list of lists
cast_info = Counter(cast_flatten)

wordcloud = WordCloud(background_color='white')
wordcloud.generate_from_frequencies(dict(cast_info.most_common(50)))

plt.figure(figsize=(15,10))
plt.title("Cast", fontsize=25, pad=20)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off");

In [ ]:

plt.xticks(fontsize=12)
plt.ylabel("Nb. of occurences", fontsize=14)
plt.title("Movie release year", fontsize=25, pad=20)
df_movies["release_year"].plot.hist(figsize=(15,5), bins=100);

In [ ]:
matrix = df_movies[["budget", "popularity", "revenue", "runtime", "vote_average", "vote_count"]].corr()

f, ax = plt.subplots(figsize=(12, 10))
plt.title("correlation between features", fontsize=20, pad=20)
plt.xticks(range(len(matrix.index)), matrix.index, fontsize=12)
plt.yticks(range(len(matrix.index)), matrix.index, fontsize=12)

# adding values
for i in range(len(matrix.index)):
    for j in range(len(matrix.index)):
        text = ax.text(j, i, round(matrix.iloc[i, j],3), ha="center", va="center", color="w", fontsize=12)
        
plt.imshow(matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar();

In [ ]:
bq_table = df_movies[["budget", "popularity", "revenue", "runtime", "vote_average", "vote_count"]]

In [ ]:
bq_table.info()

# Load the dataframe to BigQuery

In [ ]:
bq_table.head(1)

In [ ]:
test = client.get_dataset(dataset_ref='session9')

In [ ]:
test

In [ ]:
dataset_id = 'session9'

# dataset = client.create_dataset(dataset_id)  
dataset = client.get_dataset(dataset_ref=dataset_id)

In [ ]:
dataset

In [ ]:
# Optionally set explicit indices.
# If indices are not specified, a column will be created for the default
# indices created by pandas.
# df = pandas.DataFrame(records, index=pandas.Index(index, name="wikidata_id"))

table_ref = dataset.table("movie_database2")
job = client.load_table_from_dataframe(bq_table, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

In [ ]:
query = """
SELECT
  *
FROM
  `ml-session.session9.movie_database2`
WHERE budget > 0
LIMIT 100
"""
query_job = client.query(
    query,
     location="US",
)

df = query_job.to_dataframe()
df.head(5)

In [ ]:
# revenue を予測するモデルを作成
ml_query = """
CREATE OR REPLACE MODEL `session9.movie_revenue_pred`
OPTIONS (model_type='boosted_tree_regressor',
         input_label_cols=['revenue'],
         max_iterations = 10,
         tree_method = 'HIST',
         subsample = 0.85,
         enable_global_explain = TRUE
) AS
SELECT
  *
FROM
  `ml-session.session9.movie_database2`
WHERE budget > 0
LIMIT 10000
"""
query_job = client.query(
    query,
     location="US",
)
df = query_job.to_dataframe()

In [ ]:
df

In [ ]:
training_info = """
SELECT
  training_run,
  iteration,
  loss,
  eval_loss,
  duration_ms,
  learning_rate
FROM
  ML.TRAINING_INFO(MODEL `session9.movie_revenue_pred`)
ORDER BY iteration ASC
"""
client.query(training_info).to_dataframe()

In [ ]:
global_explain = """
SELECT
  *
FROM
  ML.GLOBAL_EXPLAIN(MODEL `session9.movie_revenue_pred`)
"""

client.query(global_explain).to_dataframe()

In [ ]:
explain_predict = """
SELECT *
FROM
ML.EXPLAIN_PREDICT(MODEL `session9.movie_revenue_pred`,
 (
 SELECT
   25000000 AS budget,
   14.613152 AS popularity,
   137.0 AS runtime,
   5.5 AS vote_average,
   301 AS vote_count
),
STRUCT(6 AS top_k_features))
"""
client.query(explain_predict).to_dataframe()


In [ ]:
# Delete the dataset and its contents
client.delete_dataset(dataset, delete_contents=True)

print('Deleted dataset: {}'.format(dataset.path))